In [ ]:
import pandas as pd
import numpy as np
import os
import sys
#os.chdir('C:\\Users\\OtherCat\\Documents\\Code\\Bootleg_Macro')
os.chdir('/Users/jamesbishop/Documents/Python/Bootleg_Macro')
#os.chdir('/home/imbobbilly/Documents/Coding/Bootleg_Macro')
wd = %pwd
#vbt_dir = "C:\\Users\\OtherCat\\Documents\\Code\\vbt_tax"
vbt_dir = '/Users/jamesbishop/Documents/Financial/Investment/MACRO_STUDIES/CapWars_GLI/vbt_tax'
#vbt_dir = '/home/imbobbilly/Documents/Coding/Python/vbt_tax'
fdel = os.path.sep

sys.path.append(wd)
sys.path.append(vbt_dir)
import MacroBackend as mb

In [ ]:
vams_data = pd.read_excel(vbt_dir+fdel+"42M_BTC_VAMS.xlsx", index_col=0, parse_dates=True)
vams_data = vams_data.ffill().bfill()
dataf = pd.read_excel(vbt_dir+fdel+"42M_MacWeather_BTC.xlsx", index_col=0, parse_dates=True)

In [ ]:
import vectorbt as vbt
import taxCalcs

# Assume your DataFrame is called 'data' and has columns 'BTCUSD' and 'signal'
price = vams_data['BTCUSD']
signal = vams_data['signal']

# Map signals to positions: 1 for green, 0.5 for yellow, 0 for red
signal_map = {'green': 1.0, 'yellow': 0.5, 'red': 0.0}
positions = signal.map(signal_map).fillna(1.0)

# For vectorbt, you need boolean entries/exits
entries1 = (positions.diff().fillna(0) > 0).shift(1).fillna(False).rename("entries") # Enter when position increases
exits1 = (positions.diff().fillna(0) < 0).shift(1).fillna(False).rename("exits")   # Exit when position decreases

# Create orders column: 1 for entries, -1 for exits, 0 otherwise
orders = pd.Series(0, index=positions.index)
orders[entries1] = 1
orders[exits1] = -1
orders = orders.rename("orders")
orders.iloc[0] = 1 # Set the first order to 1 to start the position

In [ ]:
def clean_orders(order_series):
    # Convert to float and set all 0s to nan
    cleaned = order_series.astype(float).copy()
    cleaned[cleaned == 0] = np.nan

    # Only keep values strictly between -1.0 and 1.0 (excluding 0, which is already nan)
    mask = cleaned.abs().between(0, 1, inclusive='both')
    cleaned[~mask] = np.nan

    # Remove consecutive repeats (keep only the first of each run)
    last_value = np.nan
    for idx in cleaned.index:
        val = cleaned.loc[idx]
        if pd.isna(val):
            continue
        if val == last_value:
            cleaned.loc[idx] = np.nan
        else:
            last_value = val

    return cleaned

def create_size_diff(signal_series):
    size_diff = pd.Series(np.nan, index=signal_series.index)
    
    # Get previous signal for transition detection
    prev_signal = signal_series.shift(1)
    
    # Yellow to Green: 1.0
    yellow_to_green = (prev_signal == 'yellow') & (signal_series == 'green')
    red_to_green = (prev_signal == 'red') & (signal_series == 'green')
    size_diff[yellow_to_green | red_to_green] = 1.0
    
    # Green to Yellow OR Red to Yellow: 0.5
    green_to_yellow = (prev_signal == 'green') & (signal_series == 'yellow')
    red_to_yellow = (prev_signal == 'red') & (signal_series == 'yellow')
    size_diff[green_to_yellow | red_to_yellow] = 0.5
    
    # Yellow to Red: 0.0
    yellow_to_red = (prev_signal == 'yellow') & (signal_series == 'red')
    green_to_red = (prev_signal == 'green') & (signal_series == 'red')
    size_diff[yellow_to_red | green_to_red] = 0.0
    
    return size_diff


#### These two cells below helped to formulate the orders but now all the data can just be loaded as below

In [ ]:
# Create a new orders series based on regime transitions in the 'signal' column
orders_regime = pd.Series(np.nan, index=signal.index, name='orders_regime')

# +1 when transitioning from yellow/red to green
green_entry = (signal == 'green') & (signal.shift(1).isin(['yellow', 'red']))
orders_regime[green_entry] = 1

# -1 when transitioning from green to yellow/red or green to red
green_exit = (signal.shift(1) == 'green') & (signal.isin(['yellow', 'red']))
orders_regime[green_exit] = 0

#Shift series a day as one could not put in a trade on the same day as a signal change.
orders_regime = orders_regime.shift(1)
# Buy on the first day
if signal.iloc[0] == "green":
    orders_regime.iloc[0] = 1
size = clean_orders(orders).replace(-1, 0)
# orders_regime = clean_orders(orders_regime)

size_diff = create_size_diff(signal).shift(1)
size_diff.iloc[0] = 1

# Combine everything into the checkit DataFrame
checkit = pd.concat([entries1, exits1, orders, size, orders_regime, size_diff, signal, price], axis=1).reset_index()
checkit.columns = ['date','entries', 'exits', 'orders', 'size', 'orders_regime', 'size_diff', 'signal', 'price']
checkit.set_index('date', inplace=True, drop=True)
checkit

In [ ]:
checkit = pd.read_excel(vbt_dir+fdel+"all_data.xlsx", index_col=0)
checkit
#checkit["macweath_2"] = checkit["macWeath_coefficient"].copy()

In [ ]:
# mw_recent = dataf.loc["2020-03-01":checkit.index[-1]].copy()
# mw_recent = mw_recent.reindex(checkit.index).ffill()
# #mw_recent["macWeath_coefficient"] = mw_recent["signal"].apply(lambda x: 1.0 if x == "green" else 0.5)
# macWeath_coeff_3 = checkit["macWeath_coefficient"].copy()
# # Properly set macWeath_coeff_2 values for each signal type
# macWeath_coeff_3.loc[mw_recent["signal"] == "red"] = 0
# macWeath_coeff_3.loc[mw_recent["signal"] == "yellow"] = 0.5
# macWeath_coeff_3.loc[mw_recent["signal"] == "green"] = 1.0
# macWeath_coeff_3

In [ ]:
# mw_recent["KISS_size2"] = mw_recent["VAMS_size"] * mw_recent["target_allocation"]
# mw_recent["KISS_size"] = mw_recent["KISS_size2"].copy()
# for i in range(len(mw_recent["KISS_size2"][1:])):
#     if mw_recent["KISS_size2"].iloc[i] == mw_recent["KISS_size2"].iloc[i-1]:
#         mw_recent["KISS_size"].iloc[i] = np.nan
#     else:
#         mw_recent["KISS_size"].iloc[i] = mw_recent["KISS_size2"].iloc[i]
# mw_recent.drop(columns=["KISS_size2"], inplace=True)
# mw_recent

In [ ]:
# for i in range(len(mw_recent)):
#     if mw_recent.loc[mw_recent.index[i], "MWR_signal"] == "red":
#         mw_recent.loc[mw_recent.index[i],"MWR_size"] = 0.25
#     elif mw_recent.loc[mw_recent.index[i], "MWR_signal"] == "yellow":
#         mw_recent.loc[mw_recent.index[i],"MWR_size"] = 0.5
#     elif mw_recent.loc[mw_recent.index[i], "MWR_signal"]== "green":
#         mw_recent.loc[mw_recent.index[i],"MWR_size"] = 1.0
    
#     if mw_recent.loc[mw_recent.index[i], "VAMS"] == "red":
#         mw_recent.loc[mw_recent.index[i],"VAMS_size"] = 0.0
#     elif mw_recent.loc[mw_recent.index[i], "VAMS"] == "yellow":
#         mw_recent.loc[mw_recent.index[i],"VAMS_size"] = 0.5
#     elif mw_recent.loc[mw_recent.index[i], "VAMS"]== "green":
#         mw_recent.loc[mw_recent.index[i],"VAMS_size"] = 1.0
# mw_recent

In [ ]:
# mw_recent = dataf.loc["2020-03-01":checkit.index[-1]].copy()
# mw_recent["macWeath_coefficient"] = mw_recent["signal"].apply(lambda x: 1.0 if x == "green" else 0.5)

# mad_sig = mw_recent["macWeath_coefficient"].copy()
# mad_sig = mad_sig.reindex(index = checkit.index, method = "ffill")
# checkit = pd.concat([checkit, mad_sig], axis=1)
# checkit["42M_KISS_Siggie"] = checkit["orders_regime"]* checkit["macWeath_coefficient"]
#checkit.drop(columns={"macWeath_coefficient"}, inplace=True)
#ordat = checkit["42M_KISS_Siggie"]

# checkit["macWeath_coefficient_2"] = checkit["signal"].apply(lambda x: 1.0 if x == "green" else 0.5 if x == "yellow" else 0.25)
# checkit["asset_2(%)"] = checkit["orders_regime"].ffill() * checkit["macWeath_coefficient_2"]
# checkit["asset_diff2(%)"] = checkit["asset_2(%)"].diff()
# checkit

# checkit.loc[checkit.index[0], "asset_diff2(%)"] = 1.0
#checkit.drop("asset_diff(%)", axis = 1,  inplace = True)
# checkit.to_excel(vbt_dir+fdel+"all_data.xlsx", index=True)

In [ ]:
# Merge the AUDUSD data with the existing checkit DataFrame
# checkit["mac_weath_only"] = macWeath_coeff_3.copy()
# checkit["mac_weath_only"].loc[checkit.index[0]] = 1
# for i in range(len(checkit[1:])):
#     if macWeath_coeff_3.loc[checkit.index[i]] == macWeath_coeff_3.loc[checkit.index[i-1]]:
#         checkit.loc[checkit.index[i], "mac_weath_only"] = np.nan
#     else:
#         pass
# checkit["mac_weath_only"]
#checkit.to_excel(vbt_dir+fdel+"all_data.xlsx", index=True)

In [ ]:
# checkit.to_excel(vbt_dir+fdel+"all_data.xlsx", index=True)

In [ ]:
# Create a dictionary to hold backtest results
# Each key is a strategy name, and the value is a dictionary to hold results
backtests = {"0: Buy & Hold": {"Orders_series": np.nan, "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "1: All in/out on signal change": {"Orders_series": size, "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "2: Long green, sell yellow/red": {"Orders_series": orders_regime, "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "3: Signal dep. sizing": {"Orders_series": size_diff, "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "4: VAMS + MacWeath": {"Orders_series": checkit["42M_KISS_Siggie"], "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "5: VAMS + MacWeath #2": {"Orders_series": checkit["asset_diff(%)"], "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "6: VAMS + MacWeath #3": {"Orders_series": checkit["asset_diff2(%)"], "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan},
             "7: Macro Weather Only": {"Orders_series": checkit["mac_weath_only"], "from_orders": np.nan, "taxed": np.nan, "tax-free": np.nan, "details": np.nan}}

In [ ]:
fees = 0.0035
init_cash = 100000

# Buy & Hold control - for comparison - benchmark
bh_pf = vbt.Portfolio.from_holding(
    price,
    init_cash=init_cash,
    freq='1D',
    fees=fees
)

# Run the strategy
full_inout = vbt.Portfolio.from_signals(
    price,
    entries=entries1,
    exits=exits1,
    init_cash=init_cash,
    freq='1D',
    fees=fees# 1% fees per trade
)

inout_ords = vbt.Portfolio.from_orders(
    price,
    size=size,
    direction="longonly",
    size_type='targetpercent',
    init_cash=init_cash,
    freq='1D',
    fees=fees # 1% fees per trade
)

less_ords = vbt.Portfolio.from_orders(
    price,
    size=orders_regime,
    direction="longonly",
    size_type='targetpercent',
    init_cash=init_cash,
    freq='1D',
    fees=fees # 1% fees per trade
)

cpf_fo = vbt.Portfolio.from_orders(
    price,
    size=size_diff,
    direction="longonly",
    size_type="targetpercent",
    init_cash=init_cash,
    freq='1D',
    fees=fees
)

full_sig = vbt.Portfolio.from_orders(
    checkit["price"],
    size=checkit["42M_KISS_Siggie"],
    direction="longonly",
    size_type="targetpercent",
    init_cash=init_cash,
    freq='1D',
    fees=fees 
)

full_sig_mod = vbt.Portfolio.from_orders(
    checkit["price"],
    size=checkit["asset_diff(%)"],
    direction="longonly",
    size_type="targetpercent",
    init_cash=init_cash,
    freq='1D',
    fees=fees 
)

full_sig_mod2 = vbt.Portfolio.from_orders(
    checkit["price"],
    size=checkit["asset_diff2(%)"],
    direction="longonly",
    size_type="targetpercent",
    init_cash=init_cash,
    freq='1D',
    fees=fees 
)

mwr_only = vbt.Portfolio.from_orders(
    checkit["price"],
    size=checkit["mac_weath_only"],
    direction="longonly",
    size_type="targetpercent",
    init_cash=init_cash,
    freq='1D',
    fees=fees 
)

# kiss = vbt.Portfolio.from_orders(
#     checkit["price"],
#     size=mw_recent["KISS_size"],
#     direction="longonly",
#     size_type="targetpercent",
#     init_cash=10000,
#     freq='1D',
#     fees=0.0035 # 1% fees per trade
# )

# Save the portfolios to a dictionary for easy access
from_holding = {
    "0: Buy & Hold": bh_pf,
    "1: All in/out on signal change": full_inout,
    "2: Long green, sell yellow/red": inout_ords,
    "3: Signal dep. sizing": less_ords,
    "4: VAMS + MacWeath": cpf_fo,
    "5: VAMS + MacWeath #2": full_sig,
    "6: VAMS + MacWeath #3": full_sig_mod,
    "7: Macro Weather Only": mwr_only
}


In [ ]:
fullstats = pd.concat([bh_pf.stats(), inout_ords.stats(), less_ords.stats(), cpf_fo.stats(), full_sig.stats(), full_sig_mod.stats(), full_sig_mod2.stats(), mwr_only.stats()], axis=1)
fullstats.columns = ['Buy & Hold (benchmark)',  'All in/out on signal change', 'Long green, sell yellow/red', "Signal dep. sizing", "VAMS + MacWeath", 
                     "VAMS + MacWeath #2", "VAMS + MacWeath #3", "Macro Weather Only"]
fullstats.to_excel(vbt_dir+fdel+"42M_MacWeather_BTC_Stats.xlsx")
fullstats

In [ ]:
returns = pd.concat([bh_pf.value(), inout_ords.value(), less_ords.value(), cpf_fo.value(), full_sig.value(), full_sig_mod.value(), full_sig_mod2.value(), mwr_only.value()], axis=1)
returns.columns = ['0: Buy & Hold (benchmark)',  '1: All in/out on signal change', '2: Long green, sell yellow/red', "3: Signal dep. sizing", "4: VAMS + MacWeath", 
                     "5: VAMS + MacWeath #2", "6: VAMS + MacWeath #3", "7: Macro Weather Only"]

# Divide all columns by the benchmark column
returns_rel = returns.div(returns['0: Buy & Hold (benchmark)'], axis=0) * 100
returns_rel -=100

In [ ]:
# title = f'Bitcoin trading using 42 MACRO VAMS Signal vs <i>hAvInG dIaMoNd hAnDs. With Tax Payments\
#     <br><sub><b>Values 2025-03-05: strategy #6: </b>{pf5tax.value().iloc[-1]:.2f} USD, <b>Strategy #6 no CGT discount: </b> {pf6tax.value().iloc[-1]:.2f} USD <b>Hodler: </b>{hodler_final_val_series.iloc[-1]:.2f} USD</sub>'
plotThese = {returns_rel.columns[i]: returns_rel[returns_rel.columns[i]] for i in range(len(returns_rel.columns))}
relatives = taxCalcs.plot_portfolios(plotThese, title = "Strategy Returns relative to the Benchmark (hodling)", return_fig = True)#"BTC: trading using 42 MACRO VAMS Signal vs hAvInG dIaMoNd hAnDs")
relatives.update_layout(
    yaxis_title="Returns relative to hOdLiNg (%)",
    yaxis_type="linear",
)
relatives.show()

In [ ]:
# relatives.write_image(vbt_dir+fdel+"relative_returns.png", scale = 3)
# relatives.write_html(vbt_dir+fdel+"relative_returns.html")

In [ ]:
# Make a copy of the DataFrame
rounded_stats = fullstats.copy()

# Define columns that should keep their original values exactly as they are
preserve_exact_columns = ['Total Trades', 'Total Closed Trades', 'Total Open Trades']

# Process the DataFrame for proper rounding
for col in rounded_stats.columns:
    for idx in rounded_stats.index:
        value = rounded_stats.loc[idx, col]
        
        # Handle date/time values
        if idx == 'Start' or idx == 'End':
            # Convert datetime to date only (strip time)
            if hasattr(value, 'date'):
                rounded_stats.loc[idx, col] = value.date()
            continue
            
        # Handle duration values
        if idx == 'Period' or idx == 'Max Drawdown Duration' or 'Duration' in idx:
            # If it's a timedelta or a string representing a duration
            if hasattr(value, 'days'):
                # Just keep days for timedeltas
                rounded_stats.loc[idx, col] = f"{value.days} days"
            elif isinstance(value, str) and 'days' in value:
                # Extract just the days portion from strings like "846 days 00:00:00"
                days = value.split(' days')[0]
                rounded_stats.loc[idx, col] = f"{days} days"
            continue
        
        # Skip exact preserved columns
        if idx in preserve_exact_columns:
            continue
        
        # Handle NaN values
        if pd.isna(value):
            continue
            
        # Round numeric values
        try:
            if isinstance(value, (int, float)) or (isinstance(value, str) and value.replace('.', '', 1).isdigit()):
                num_value = float(value)
                rounded_stats.loc[idx, col] = round(num_value, 2)
        except (ValueError, TypeError):
            # Keep the original value if conversion fails
            pass

# Display the rounded DataFrame
rounded_stats

In [ ]:
rounded_stats.to_markdown()

In [ ]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

# Add each portfolio's value curve
portfolios = {
    '#0: HoDlEr': bh_pf,
    '#1: VAMS signal change': inout_ords,
    '#2: Long green, sell yellow/red': less_ords,
    '#3: VAMS dep. size': cpf_fo,
    '#4: VAMS + MWR': full_sig,
    '#5: VAMS + MWR #2': full_sig_mod,
    '#6: VAMS + MWR#3': full_sig_mod2,
    '#7: MWR  Only': mwr_only,
}

for name, pf in portfolios.items():
    fig.add_trace(
        go.Scatter(
            x=pf.value().index,
            y=np.round(pf.value().values, 0),
            name=name,
            mode='lines',
            line=dict(width=1.5)
        )
    )

# Update layout with horizontal legend at bottom
fig.update_layout(
    title='BTC Trading using 42 MACRO VAMS and Macro Weather regime',
    yaxis_title='Portfolio Value ($)',
    template='plotly_white',
    width=1100,
    height=600,
    yaxis_type='log',
    showlegend=True,
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    )
)

# Show the figure
fig.show()

In [ ]:
#fig.write_html(vbt_dir+fdel+"42Macro_VAMSStrats.html")

In [ ]:
# audusd = mb.Pull_Data.dataset()
# audusd.get_data("tv", "USDAUD", exchange_code="FX_IDC", start_date="2020-03-01", end_date="2025-03-05", data_freq="1d", dtype="close")

In [ ]:
# checkit["audusd"] = audusd.data["2020-03-01":"2025-03-05"].resample("D").ffill().bfill()
# df = pd.concat([price, aud], axis = 1).bfill()
# aud = df["USDAUD"]
# aud.to_excel(vbt_dir+fdel+"AUDUSD.xlsx", sheet_name="AUDUSD", index=True)
# aud = pd.read_excel(vbt_dir+fdel+"AUDUSD.xlsx", index_col=0, parse_dates=True)

### Payable tax formula: 

$Tax_{net} = ((I_{g} - D) - Threshold_{\beta}) \times Rate_{\beta} + Base_{\beta}$

**Where:**
- $I_{g}$ is gross taxable income.
- $D$ are eligible tax deductions gross total.
- $(I_{g} - D)$ is net taxable income.
- $\beta$ idicates the tax bracket applicable.
- $Threshold_{\beta}$ is minimum threshold for tax bracket
- $Rate_{\beta}$ is tax rate for bracket (%)
- $Base_{\beta}$ is base tax amount for bracket.


## New tax calculation backtest..

```python
def run_backtest_with_tax(
    price: pd.Series,
    orders: pd.Series,
    aud: pd.Series,
    init_cash: float,
    income: Union[float, list],
    deductions: Union[float, list] = 0,
    tax_rates: pd.DataFrame = None,
    freq: str = '1D'
) -> vbt.Portfolio:
```

In [ ]:
years = price.index.year.unique()
income = pd.Series([91000, 93000, 96000, 103000, 112000, 117000], index=years)
cgt_list = pd.Series([1.0, 0.5, 1.0, 1.0, 1.0, 1.0], index=years)
deductions = pd.Series([5000, 4200, 2200, 5000, 6000, 7500], index=years)
years

In [ ]:
# Create a salary series with 3% annual growth
base_salary_hi = 181000  # Starting salary
base_salary_lo = 90000   # Starting salary for lower income
growth_rate = 0.03   # 3% annual growth
num_years = len(years)
salaries = np.array([base_salary_hi * (1 + growth_rate)**i for i in range(num_years)])
salaries = salaries.round().tolist()
salaries

In [ ]:
pfnotax, pftax, details = taxCalcs.run_backtest_with_tax(price, checkit["orders_regime"], checkit["audusd"], init_cash, salaries)

In [ ]:
# df = details["taxed"]["all_trades"].reset_index(drop=True)
# df

In [ ]:
tax_details = details["taxed"]["tax_details"]
tax_details

In [ ]:
#full_sig_mod2.trades.records_readable

In [ ]:
index = pd.date_range(start="2020-01-01", end="2025-01-01", freq="YS")
index 

In [ ]:
import plotly.express as px

# Get the tax details from the dictionary
tax_details = details["taxed"]["tax_details"]

# Create a bar chart of CGT tax paid in USD
fig = px.bar(
    x=index,
    y=tax_details.loc["cgt_tax_paid_(usd)"].values,
    labels={"x": "Financial Year", "y": "CGT Tax Paid (USD)"},
    title="Capital Gains Tax Paid Per Financial Year (USD)",
    text_auto='.2f'
)

# Improve layout
fig.update_layout(
    xaxis_title="Financial Year",
    yaxis_title="CGT Tax Paid (USD)",
    template="plotly_white",
    width=800,
    height=500
)

# Display the figure
fig.show()

In [ ]:
def plot_timeseries_with_markers(timeseries, marker_dates_dict=None, figsize=(12,6)):
    """Plot a timeseries with optional vertical lines at specified dates
    
    Args:
        timeseries: pandas Series with DatetimeIndex
        marker_dates_dict: dict where keys are colors and values are DatetimeIndex of dates to mark
        figsize: tuple specifying figure size
    """
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(timeseries.index, timeseries.values)
    
    if marker_dates_dict:
        for color, dates in marker_dates_dict.items():
            for date in dates:
                ax.axvline(x=date, color=color, alpha=0.3, linestyle='--')
    
    plt.grid(True, alpha=0.3)
    return fig



#FUNCTIONS #########################################################################################################################

## Benchmark (hodler) portfolio tax details

In [ ]:
## Hodler CGT tax calculation 
## PARAMETERS:
year = 2025
cgt_rate_hodler  = 0.5 # 50% CGT discount for hodlers in Australia that held > 1 yr
fixed_cgt_rate = 0.45  # Set to a fixed CGT rate if applicable, otherwise None

#Derived parameters
aud_mar5 = checkit["audusd"].iloc[-1]  #AUD/USD exchange rate on March 5, 2025
final_sale_fee = bh_pf.value().iloc[-1]*fees # Trading fee on final sale for hodler
hodler_sell_mar5 = bh_pf.value().iloc[-1] - init_cash -  final_sale_fee # Total capital gains from hodling BTC, selling mar5 2025
inaud = hodler_sell_mar5*checkit["audusd"].iloc[-1] # Convert to AUD
salary_income_2025 = salaries[-1] - deductions.iloc[-1]  # Salary income for 2025 after deductions
hodler_cg = inaud * cgt_rate_hodler  # Capital gains for hodler with 50% CGT discount
total_income = hodler_cg + salary_income_2025  # Total income for 2025 (50% of CGT + salary income after deductions)

print(f"Hodler capital gains: {hodler_sell_mar5} USD, {inaud} AUD,\nSalary income: {salary_income_2025} AUD\n\
      fee on final sale: {final_sale_fee} USD, {final_sale_fee*aud_mar5} AUD")
print(f"Capital gains for hodler with 50% CGT discount: {hodler_cg} AUD, {hodler_cg/aud_mar5} USD")
print(f"Total income: {total_income} AUD, {total_income/aud_mar5} USD")
print("Fraction of total income coming from cap. gains: {:.2f}%".format((hodler_cg/total_income)*100))
bracket = taxCalcs.determine_tax_bracket(total_income, year)
taxDeets = taxCalcs.defaultTaxRates()
tax_rates = taxDeets[0]
brackets = taxDeets[1]
bracket_df = brackets[bracket]  #This is the tax rates for the given tax bracket for years 2020 to 2025

print(f"Tax bracket for 2025: {bracket}\n")

base_tax = bracket_df.loc[year, "base_tax_aud"]
marginal_rate = bracket_df.loc[year, "tax_rate_(%)"]
print(f"Base tax: {base_tax}, Marginal rate: {marginal_rate}")

tax_that_Year = (total_income - bracket_df.loc[year, "bracket_min_aud"])*(marginal_rate/100) + base_tax
print(f"Total tax for 2025: {tax_that_Year} AUD, {tax_that_Year/aud_mar5} USD")

if fixed_cgt_rate is not None:
      cgt_hodler = (hodler_cg * fixed_cgt_rate)  # Fixed CGT rate applied to hodler capital gains
else:
      cgt_hodler = (hodler_cg/total_income)*tax_that_Year

final_hodler_value = bh_pf.value().iloc[-1] - cgt_hodler/checkit["audusd"].iloc[-1] #Final value of hodler portfolio after CGT tax applied (USD)
print(f"Final hodler value: {final_hodler_value} USD\nCGT tax paid: {cgt_hodler} AUD, {cgt_hodler/aud_mar5} USD")
print("Hodler final return after CGT tax: {:.2f}%".format(((final_hodler_value/init_cash) - 1)*100))
print("CGT as percentage of capital gains: {:.2f}%".format((cgt_hodler/inaud)*100))

# Create a Series for the final value of the hodler portfolio with CGT applied
hodler_final_val_series = bh_pf.value().copy()
hodler_final_val_series.iloc[-1] -= cgt_hodler/checkit["audusd"].iloc[-1]

bench_sharpe = hodler_final_val_series.pct_change().mean() / bh_pf.value().pct_change().std()
bench_sharpe *= np.sqrt(365)  # Annualize the Sharpe ratio
print(f"Sharpe ratio for hodler: {bench_sharpe:.2f}")

In [ ]:
bracket_df

In [ ]:
tax_rates.sort_values("bracket").groupby("bracket")

In [ ]:
details["taxed"]["tax_details"].loc["cgt_tax_paid_(usd)"].sum()

In [ ]:
pf2, pf2tax, details2 = taxCalcs.run_backtest_with_tax(price, checkit["orders_regime"], checkit["audusd"], init_cash, salaries, special_conditions={"constant_cgt_discount": 1.0, "fixed_cgt_rate": 0.45})

In [ ]:
details2["taxed"]["tax_details"]

In [ ]:
pf3, pf3tax, details3 = taxCalcs.run_backtest_with_tax(price, checkit["42M_KISS_Siggie"], checkit["audusd"], init_cash, salaries)#, special_conditions={"constant_cgt_rate": 1.0})

In [ ]:
pf4, pf4tax, details4 = taxCalcs.run_backtest_with_tax(price, checkit["asset_diff(%)"], checkit["audusd"], init_cash, salaries)
pf5, pf5tax, details5 = taxCalcs.run_backtest_with_tax(price, checkit["asset_diff2(%)"], checkit["audusd"], init_cash, salaries, deductions=[0, 0, 0, 0, 0, 0])#, special_conditions={"constant_cgt_rate": 1.0})

In [ ]:
details5["taxed"]["tax_details"]

In [ ]:
pf6, pf6tax, details6 = taxCalcs.run_backtest_with_tax(price, checkit["asset_diff2(%)"], checkit["audusd"], init_cash, salaries, special_conditions={"constant_cgt_discount": 1.0, "fixed_cgt_rate": 0.45})

In [ ]:
pf7, pf7tax, details7 = taxCalcs.run_backtest_with_tax(price, checkit["mac_weath_only"], checkit["audusd"], init_cash, salaries)#, special_conditions={"constant_cgt_rate": 1.0})

In [ ]:
details5["taxed"]["tax_details"]

In [ ]:
# "NoTaxCalc": less_ords, "1 BT per FY (no tax)": pfnotax, "1 BT per FY (taxed)": pftax,
# </i><br><sub>Tax payment simulation included for Australian who earnt 90k AUD in 2020 with 3% wage growth. CGT paid for 2025 on last day of backtest.</sub><br>
title = f'Bitcoin trading using 42 MACRO VAMS Signal vs <i>hAvInG dIaMoNd hAnDs. With Tax Payments\
    <br><sub><b>Values 2025-03-05: strategy #6: </b>{pf5tax.value().iloc[-1]:.2f} USD, <b>Strategy #6 no CGT discount: </b> {pf6tax.value().iloc[-1]:.2f} USD <b>Hodler: </b>{hodler_final_val_series.iloc[-1]:.2f} USD</sub>'
title2 = f'High Income Scenario: top Tax Bracket, initial investment = $10k USD\
    <br><sub><b>Values 2025-03-05: strategy #6: </b>{pf5tax.value().iloc[-1]:.2f} USD, <b>Strategy #6 no CGT discount: </b> {pf6tax.value().iloc[-1]:.2f} USD <b>Hodler: </b>{hodler_final_val_series.iloc[-1]:.2f} USD</sub>'
title3 = f'Wealthy Guy Scenario: top Tax Bracket, initial investment = $100k USD\
    <br><sub><b>Values 2025-03-05: strategy #6: </b>{pf5tax.value().iloc[-1]:.2f} USD, <b>Strategy #6 no CGT discount: </b> {pf6tax.value().iloc[-1]:.2f} USD <b>Hodler: </b>{hodler_final_val_series.iloc[-1]:.2f} USD</sub>'
compfig = taxCalcs.plot_portfolios({"hOdLeR, tax": hodler_final_val_series, "#3, no tax": pfnotax.value(),
                                    "#2, tax": pftax.value(), "#2, tax, no CGT discount": pf2tax.value(), "#4, tax": pf3tax.value(),
                                   "#6, tax": pf5tax.value(), "#6, tax, no CGT discount": pf6tax.value(), "#7, tax": pf7tax.value()},
                                   title = title3, return_fig = True)#"BTC: trading using 42 MACRO VAMS Signal vs hAvInG dIaMoNd hAnDs")
compfig.show()

In [ ]:
compfig.write_html(vbt_dir+fdel+"WithCGTPayments3.html")
compfig.write_image(vbt_dir+fdel+"WithCGTPayments3.png", scale = 3)

In [ ]:
details.keys()

In [ ]:
details["non_taxed"]["stats"]

In [ ]:
names = ['Strat #0: Hodler', 'Strat #2: CGT discount', 'Strat #2: No CGT discount', "Strat #4, CGT discount", 
         "Strat #5, CGT discount", "Strat #6, CGT discount", "Strat #6, No CGT discount", "Strat #7, CGT discount"]
final_vals = pd.DataFrame([hodler_final_val_series.iloc[-1], pftax.value().iloc[-1], pf2tax.value().iloc[-1], pf3tax.value().iloc[-1], pf4tax.value().iloc[-1], 
              pf5tax.value().iloc[-1], pf6tax.value().iloc[-1], pf7tax.value().iloc[-1]],
              index=names, columns=['Final Post-tax Value (USD)']
            )
final_vals["Final Tax-free Value (USD)"] = [bh_pf.value().iloc[-1], pfnotax.value().iloc[-1], pf2.value().iloc[-1], pf3.value().iloc[-1], 
                                                 pf4.value().iloc[-1], pf5.value().iloc[-1], pf6.value().iloc[-1], pf7.value().iloc[-1]]
final_vals["Sharpe (taxed pf)"] = [bench_sharpe, details["taxed"]["stats"]["Sharpe Ratio"], details2["taxed"]["stats"]["Sharpe Ratio"], details3["taxed"]["stats"]["Sharpe Ratio"],
                                     details4["taxed"]["stats"]["Sharpe Ratio"], details5["taxed"]["stats"]["Sharpe Ratio"], details6["taxed"]["stats"]["Sharpe Ratio"],
                                     details7["taxed"]["stats"]["Sharpe Ratio"]]
final_vals

In [ ]:
lo_sharpe = less_ords.stats()["Sharpe Ratio"]
cp_sharpe = cpf_fo.stats()["Sharpe Ratio"]
full_sharpe = full_sig.stats()["Sharpe Ratio"]
full_sharpe2 = full_sig_mod.stats()["Sharpe Ratio"]
full_sharpe3 =full_sig_mod2.stats()["Sharpe Ratio"]
mwr_sharpe = mwr_only.stats()["Sharpe Ratio"]

In [ ]:
full_sig_mod2.stats().index

In [ ]:
# "NoTaxCalc": less_ords, "1 BT per FY (no tax)": pfnotax, "1 BT per FY (taxed)": pftax,
#<br><sub> Macro weather regime (MWR) adds coefficient to position size. MWR bullish = no reduction to position size,when neutral/bearish, position size reduced.

# Add each portfolio's value curve
portfolios = {
    '#0: HoDlEr': bh_pf,
    '#1: VAMS signal change': inout_ords,
    '#2: Long green, sell yellow/red': less_ords,
    '#3: VAMS dep. size': cpf_fo,
    '#4: VAMS + MWR': full_sig,
    '#5: VAMS + MWR #2': full_sig_mod,
    '#6: VAMS + MWR #3': full_sig_mod2,
    '#7: MWR  Only': mwr_only,
}

title = f"BTC: trading using 42 MACRO VAMS and Macro Weather Regime Overlay vs <i>hAvInG dIaMoNd hAnDs</i> <br> <sub> Strategy #6 perfoms best. \
<b>Final value strategy: </b>{full_sig_mod2.value().iloc[-1]:.2f} USD, <b>final value hodler: </b>{bh_pf.value().iloc[-1]:.2f} USD</sub>"
compfig3 = taxCalcs.plot_portfolios({f'#0: HoDlEr, sharpe ratio: {bh_pf.stats()["Sharpe Ratio"]:.2f}': bh_pf.value(), 
                                     f"#2: long green only, sharpe: {lo_sharpe:.2f}": less_ords.value(),
                                    f'#3: VAMS dep. size, sharpe: {cp_sharpe:.2f}': cpf_fo.value(), 
                                    f'#4: VAMS + MWR, sharpe: {full_sharpe:.2f}': full_sig.value(),
                                    f'#5: VAMS + MWR #2, sharpe: {full_sharpe2:.2f}': full_sig_mod.value(),
                                    f'#6: VAMS + MWR #3, sharpe: {full_sharpe3:.2f}': full_sig_mod2.value(),
                                    f'#7: MWR only, sharpe: {mwr_sharpe:.2f}': mwr_only.value()},
                                   title = title, return_fig = True)#"BTC: trading using 42 MACRO VAMS Signal vs hAvInG dIaMoNd hAnDs")
compfig3.show()


In [ ]:
##compfig3.write_image(vbt_dir+fdel+"strats_with_sharpe.png", scale=3)

In [ ]:
#compfig3.write_html(vbt_dir+fdel+"strats_with_sharpe.html", include_plotlyjs=True, default_height=800, default_width="100%")

In [ ]:
title = f"Bitcoin trading strategies using 42 MACRO signals: high-fee non-KYC scenario \
<br><sub> <b>Final value strategy: </b>{full_sig_mod2.value().iloc[-1]:.2f} USD, <b>final value hodler: </b>{bh_pf.value().iloc[-1]:.2f} USD</sub>"
compfigga = taxCalcs.plot_portfolios({f'#0: HoDlEr, sharpe ratio: {bh_pf.stats()["Sharpe Ratio"]:.2f}': bh_pf.value(), 
                                     f"#2: long green only, sharpe: {lo_sharpe:.2f}": less_ords.value(),
                                    f'#3: VAMS dep. size, sharpe: {cp_sharpe:.2f}': cpf_fo.value(), 
                                    f'#4: VAMS + MWR, sharpe: {full_sharpe:.2f}': full_sig.value(),
                                    f'#5: VAMS + MWR #2, sharpe: {full_sharpe2:.2f}': full_sig_mod.value(),
                                    f'#6: VAMS + MWR #3, sharpe: {full_sharpe3:.2f}': full_sig_mod2.value(),
                                    f'#7: MWR only, sharpe: {mwr_sharpe:.2f}': mwr_only.value()},
                                   title = title, return_fig = True)#"BTC: trading using 42 MACRO VAMS Signal vs hAvInG dIaMoNd hAnDs")
compfigga.show()

In [ ]:
compfigga.write_image(vbt_dir+fdel+"highFeeScenario.png", scale=3)
compfigga.write_html(vbt_dir+fdel+"highFeeScenario.html")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from matplotlib.colors import to_rgba
from matplotlib.patches import Patch
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# Create the figure and the primary axes
figga, ax = plt.subplots(figsize=(12, 6))

# --- Add Background Colors ---
signal_colors = {
    'green': to_rgba('green', 0.5),
    'yellow': to_rgba('gold', 0.5),
    'red': to_rgba('red', 0.5),
    'None': to_rgba('white', 1) # Transparent for "None"
}

# Find where the signal changes
signal_changes = vams_data['signal'] != vams_data['signal'].shift(1)
change_indices = vams_data.index[signal_changes].tolist()
# Ensure the first and last dates are included to cover the whole range
if vams_data.index[0] not in change_indices:
    change_indices.insert(0, vams_data.index[0])
if vams_data.index[-1] not in change_indices:
    change_indices.append(vams_data.index[-1])

# Add colored vertical spans (backgrounds)
for i in range(len(change_indices) - 1):
    start_date = change_indices[i]
    end_date = change_indices[i+1]
    # Get the signal value for this period (use the start date's signal)
    signal_value = vams_data.loc[start_date, 'signal']
    # Get the color for this signal
    color = signal_colors.get(signal_value, 'white') # Default to white if signal not found
    
    # Add the vertical span using axvspan
    ax.axvspan(start_date, end_date, facecolor=color, alpha=0.5, zorder=0) 

# Set the label and limits for the left y-axis
ax.set_ylabel('USD', color='black', fontsize=12)
ax.tick_params(axis='y', labelcolor='black')

# Plot the 'BTCUSD' column on the left axis
btc_line = ax.plot(vams_data.index, vams_data['BTCUSD'], color='black', label='BTC (USD))', linewidth=1.5)[0] # Ensure price is above background

# Set the label for the right y-axis and use log scale
ax.set_ylabel('BTC Price ($)', color='black', fontsize=12)
ax.set_yscale('log') # Use logarithmic scale for price
ax.tick_params(axis='y', labelcolor='black')

# --- Formatting ---

# Improve date formatting on the x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3)) # Ticks every 6 months # Auto-rotate date labels

# Add a title
plt.title('42 Macro Volatility Adjusted Momentum Signal (VAMS) for BTC-USD', fontsize=14)

# Create proxy artists for the legend
legend_elements = [
    Patch(facecolor=signal_colors['green'], label='Bullish VAMS'),
    Patch(facecolor=signal_colors['yellow'], label='Neutral VAMS'),
    Patch(facecolor=signal_colors['red'], label='Bearish VAMS'),
    btc_line
]

# Add the legend with the proxy patches
ax.legend(handles=legend_elements, loc='upper left')
# Improve date formatting on the x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
# Set the rotation of the labels to 45 degrees
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right') 

# Add grid lines for better readability (optional)
ax.grid(True, axis='y', linestyle='--', alpha=0.6) # Grid for left axis
ax.grid(True, axis='y', linestyle=':', alpha=0.6) # Grid for right axis (price)
ax.margins(0.02, 0.02)
# Adjust layout to prevent labels overlapping
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
def plot_signal_colored_timeseries(
    data,
    price_col,
    signal_col,
    signal_colors=None,
    figsize=(12, 6),
    title="Signal Colored Time Series",
    use_log_scale=True,
    date_format='%Y-%m',
    date_interval=3,
    legend_loc='upper left'
):
    """
    Create a time series plot with colored backgrounds based on signal values.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame with DatetimeIndex containing price and signal data
    price_col : str
        Column name for the price data to be plotted
    signal_col : str
        Column name for the signal data that determines background colors
    signal_colors : dict, optional
        Dictionary mapping signal values to colors (RGBA or color names)
        Default: {'green': (0, 0.5, 0, 0.5), 'yellow': (1, 0.84, 0, 0.5), 'red': (1, 0, 0, 0.5)}
    figsize : tuple, optional
        Figure size as (width, height) in inches
    title : str, optional
        Plot title
    use_log_scale : bool, optional
        Whether to use logarithmic scale for price axis
    date_format : str, optional
        Format string for date labels on x-axis
    date_interval : int, optional
        Interval in months between date ticks
    legend_loc : str, optional
        Location of the legend
        
    Returns:
    --------
    fig, ax : matplotlib figure and axes objects
    """
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    from matplotlib.colors import to_rgba
    from matplotlib.patches import Patch
    import pandas as pd
    import numpy as np
    
    # Make a copy of the data to avoid modifying the original
    data_copy = data.copy()
    
    # Fill NaN values with 'None' string
    data_copy[signal_col] = data_copy[signal_col].fillna('None')
    
    # Default signal colors if none provided
    if signal_colors is None:
        signal_colors = {
            'green': to_rgba('green', 0.5),
            'yellow': to_rgba('gold', 0.5),
            'red': to_rgba('red', 0.5),
            'None': to_rgba('white', 1)  # Transparent for "None"
        }
    else:
        # Convert any string colors to rgba
        for key, value in signal_colors.items():
            if isinstance(value, str):
                signal_colors[key] = to_rgba(value, 0.5)
    
    # Create the figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Find where the signal changes
    signal_changes = data_copy[signal_col] != data_copy[signal_col].shift(1)
    change_indices = data_copy.index[signal_changes].tolist()
    
    # Ensure the first and last dates are included
    if data_copy.index[0] not in change_indices:
        change_indices.insert(0, data_copy.index[0])
    if data_copy.index[-1] not in change_indices:
        change_indices.append(data_copy.index[-1])
    
    # Add colored vertical spans (backgrounds)
    used_signals = set()
    
    for i in range(len(change_indices) - 1):
        start_date = change_indices[i]
        end_date = change_indices[i+1]
        
        # Get signal value for this period and convert to string
        signal_value = data_copy.loc[start_date, signal_col]
        signal_key = str(signal_value)
        
        # Get color (default to white if not found)
        color = signal_colors.get(signal_key, to_rgba('white', 0.5))
        
        # Add the vertical span
        ax.axvspan(start_date, end_date, facecolor=color, alpha=0.5, zorder=0)
        
        # Track used signals for legend
        used_signals.add(signal_key)
    
    # Convert the index to datetime64 and the price column to numeric values
    x_values = data_copy.index.to_numpy()
    y_values = pd.to_numeric(data_copy[price_col], errors='coerce').to_numpy()
    
    # Plot the price data directly with numpy arrays to avoid category converter issues
    price_line, = ax.plot(
        x_values,
        y_values,
        color='black',
        label=price_col,
        linewidth=1.5,
        zorder=5  # Ensure line is above background
    )
    
    # Set up logarithmic scale if requested
    if use_log_scale:
        ax.set_yscale('log')
    
    # Format the x-axis dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter(date_format))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=date_interval))
    
    # Rotate date labels
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Set labels and title
    ax.set_ylabel(f'{price_col}', color='black', fontsize=12)
    ax.set_title(title, fontsize=14)
    
    # Create legend elements for used signals
    legend_elements = [Patch(facecolor=signal_colors[s], label=s) for s in used_signals if s in signal_colors]
    legend_elements.append(price_line)
    
    # Add the legend
    ax.legend(handles=legend_elements, loc=legend_loc)
    
    # Add grid lines
    ax.grid(True, axis='y', linestyle='--', alpha=0.6)
    
    # Adjust layout
    plt.tight_layout()
    
    return fig, ax

In [ ]:
figur,ax = plot_signal_colored_timeseries(
    dataf["2010-01-01":],
    price_col='BTCUSD',
    signal_col='signal',
    signal_colors={
        'green': to_rgba('green', 0.5),
        'yellow': to_rgba('gold', 0.5),
        'red': to_rgba('red', 0.5),
        'None': to_rgba('white', 1)
    },
    title="42 Macro Weather Signal",
    use_log_scale=True,
    date_format='%Y-%m',
    date_interval=12,
    legend_loc='upper left'
)


In [ ]:
figur.savefig(vbt_dir+fdel+"MacroWeatherBTC.png", dpi=300, bbox_inches='tight')

In [ ]:
mw_recent = dataf["2020-03-01":]
mw_recent

In [ ]:
figga.savefig("42macro_vams_signals.png", dpi=300, bbox_inches='tight')

In [ ]:
backtests["1: All in/out on signal change"].keys()

In [ ]:
backtests.keys()

## Put all tests into a single dictionary

In [ ]:
special_conditions={"constant_cgt_discount": None, "fixed_cgt_rate": 0.45}  #Special conditions for tax calculations, e.g. constant CGT discount or fixed CGT rate 1.0 = no discount, 1.0 = full taxation

# All backtests, standard from_orders and then taxed and tax-free from my looped function. tax-free should be the same as the from_orders.
for key in from_holding.keys():
    if key == "0: Buy & Hold":
        backtests[key]["from_orders"] = bh_pf
        backtests[key]["taxed"], backtests[key]["tax-free"] = (hodler_final_val_series, np.nan)
        backtests[key]["details"] = bh_pf.stats()
    else:
        backtests[key]["from_orders"] = from_holding[key]
        backtests[key]["taxed"], backtests[key]["tax-free"], backtests[key]["details"] = taxCalcs.run_backtest_with_tax(price, backtests[key]["Orders_series"], checkit["audusd"], init_cash, salaries, trading_fees=fees, special_conditions=special_conditions)

In [ ]:
backtests["0: Buy & Hold"]["details"]

In [ ]:
taxed_bh_stats = taxCalcs.calculate_performance_stats(hodler_final_val_series, periods_per_year=365)
for key in taxed_bh_stats.keys():
    backtests["0: Buy & Hold"]["details"].loc[key] = taxed_bh_stats[key]
#backtests["0: Buy & Hold"]["details"]["Total Fees Paid"] += final_sale_fee  # Add the final sale fee to the total fees paid
backtests["0: Buy & Hold"]["details"]

In [ ]:
# Exclude specific keys
exclude_keys = ["0: Buy & Hold"]
col_names = [key for key in backtests.keys() if key not in exclude_keys]
taxed_all_stats = pd.concat([backtests[key]["details"]["taxed"]["stats"] for key in backtests.keys() if key not in exclude_keys], axis=1)
taxed_all_stats.columns = col_names
taxed_all_stats = pd.concat([backtests["0: Buy & Hold"]["details"].rename("0: Buy & Hold"), taxed_all_stats], axis = 1)
taxed_all_stats.to_excel(vbt_dir+fdel+"tax_bracket_A.xlsx", sheet_name="taxed", index=True)

In [ ]:
backtests["6: VAMS + MacWeath #3"]["details"]["taxed"]["tax_details"]

In [ ]:
backtests["2: No CGT discount"] = {
    "Orders_series": checkit["asset_diff(%)"],
    "from_orders": pf2,
    "taxed": pf2tax,
    "tax-free": pf2.value(),
    "details": details2}
backtests["6: No CGT discount"] = {
    "Orders_series": checkit["asset_diff2(%)"],
    "from_orders": pf6,
    "taxed": pf6tax,
    "tax-free": pf6.value(),
    "details": details6}


In [ ]:
full_stats = pd.DataFrame()
for key in backtests.keys():
    if "taxed" not in backtests[key]["details"].keys():
        pass
    else:
        full_stats = pd.concat([full_stats, backtests[key]["details"]["taxed"]["stats"].rename(key)], axis = 1)
full_stats.to_excel(vbt_dir+fdel+"fin_stats.xlsx", sheet_name="taxed", index=True)

In [ ]:
av_cgts = {}
for key in backtests.keys():

    if "taxed" not in list(backtests[key]["details"].keys()) and key != "0: Buy & Hold":
        av_cgts[key] = "Calculate manually"
    elif key == "0: Buy & Hold":
        cgt_av = (cgt_hodler/inaud)*100 
        av_cgts[key] = cgt_av  #This is the CGT paid as a percentage of the capital gains for the hodler portfolio
        print(f"{key}: Total PnL (AUD): {inaud}, CGT paid (AUD): {cgt_hodler}, CGT as % of PnL: {cgt_av:.2f}%")
    else:
        cgt_paid = backtests[key]["details"]["taxed"]["tax_details"].loc["cgt_tax_(aud)"].sum()
        pnl_total = backtests[key]["details"]["taxed"]["tax_details"].loc["closed_PnL_(AUD)"].sum()
        cgt_av = (cgt_paid/pnl_total)*100
        av_cgts[key] = cgt_av
        print(f"{key}: Total PnL (AUD): {pnl_total}, CGT paid (AUD): {cgt_paid}, CGT as % of PnL: {cgt_av:.2f}%")

In [ ]:
1423059.175580906/aud_mar5

In [ ]:
backtests["6: VAMS + MacWeath #3"]["details"]["taxed"]["tax_details"]

In [ ]:
# Method 1: Dictionary comprehension with key filtering
subset_keys = ['0: Buy & Hold', '2: Long green, sell yellow/red', '2: No CGT discount', '6: VAMS + MacWeath #3', '6: No CGT discount']  # specify which keys you want
summary_ones = pd.Series({k: av_cgts[k] for k in subset_keys if k in av_cgts}, name="Average CGT paid as % of PnL")
summary_ones

In [ ]:
backtests["6: VAMS + MacWeath #3"]["details"]["taxed"]["tax_details"].to_excel(vbt_dir+fdel+"tax_details_A.xlsx", sheet_name="tax_details", index=True)

In [ ]:
details2["taxed"]["tax_details"]